# Аугументация датасета, состоящего из данных с сайтов: Проза.ру, ЛитПричал, Брифли, Литрес

In [76]:
import re

import pandas as pd
import nltk
nltk.download("punkt")

from nltk.tokenize import sent_tokenize

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/veronika_steklo/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [41]:
dataset = pd.read_csv("../data/all_data.csv")

In [42]:
dataset.describe()

,title,text
count,31122,31101
unique,28839,30640
top,латырь-камень бел-горюч... сказка?,; ; «энцикловикипедия»- -имена. посвящается ми...
freq,38,38


In [43]:
dataset.dropna(inplace=True)

In [44]:
dataset.describe()

,title,text
count,31101,31101
unique,28818,30640
top,из жизни графомана,; ; «энцикловикипедия»- -имена. посвящается ми...
freq,38,38


In [45]:
dataset = dataset[dataset.text != ""].reset_index(drop=True)
dataset.describe()

,title,text
count,31101,31101
unique,28818,30640
top,из жизни графомана,; ; «энцикловикипедия»- -имена. посвящается ми...
freq,38,38


In [46]:
dataset[dataset.duplicated(keep=False)]

,title,text
28,латырь-камень бел-горюч... сказка?,домашняя диссертация любознательного обывателя...
40,на распутина мессмера серебряного века,"""распутин в кровавом поту молился и спасал"" пи..."
146,энцикловикипедия. топонимы,; «энцикловикипедия». топонимы. история соврем...
251,три поколения швондеров и мы выбираем...,"в повести булгакова ""собачье сердце"" есть домо..."
258,"просыпаюсь, а ты рядом! дню семьи","постоянное место публикации: сборник ""о любви..."
...,...,...
30682,таёжный глаз шерхана с гарью каина,"она заступалась за коллективного папика, подош..."
30871,энцикловикипедия. имена,; ; «энцикловикипедия»- -имена. посвящается ми...
30970,"просыпаюсь, а ты рядом! дню семьи","постоянное место публикации: сборник ""о любви..."
31011,корнеслов. история слов,; ; -история слов. «энцикловикипедия» посвящае...


In [47]:
dataset[dataset.text == "описание отсутствует"]

,title,text


In [48]:
dataset = dataset.drop_duplicates()

In [49]:
dataset.describe()

,title,text
count,30679,30679
unique,28818,30640
top,***,­
freq,20,19


In [50]:
dataset.loc[195].text

'\xad'

In [51]:
dataset = dataset[dataset.text != "\xad"].reset_index(drop=True)

In [52]:
dataset.describe()

,title,text
count,30660,30660
unique,28800,30639
top,***,"стихотворение ищите в рецензии, поскольку тата..."
freq,20,4


In [53]:
dataset = dataset[~dataset.duplicated(subset=["text"], keep=False)]

In [54]:
dataset.describe()

,title,text
count,30623,30623
unique,28763,30623
top,***,­­… сказки осенних вечеров ... монохромные во...
freq,20,1


In [56]:
dataset.reset_index(drop=True, inplace=True)

In [57]:
dataset

,title,text
0,время - октябрь...,­­… сказки осенних вечеров ... монохромные во...
1,переписка,"здравствуйте, вы установили мессенджер max, по..."
2,айвенго (спектакль) часть 3-я. ричард – львино...,радиоспектакль по роману.англия времен царство...
3,калевала,где можно найти наиболее полный литературный о...
4,янки из коннектикута при дворе короля артура,"новый, адаптированный перевод алексея козлова ..."
...,...,...
30618,что бывало,борис житков увлекался астрономией и фотографи...
30619,сон макара,"«этот сон видел бедный макар, который загнал с..."
30620,финал. золотое солнце правосудия,раннее утро предвещало жаркий день. солнечные ...
30621,святочные истории,святки – удивительный период времени: один год...


In [58]:
def split_text_into_chunks(text, sentences_per_chunk=3):
    sentences = sent_tokenize(text, language="russian")
    chunks = []
    for i in range(0, len(sentences), sentences_per_chunk):
        chunk = " ".join(sentences[i:i+sentences_per_chunk])
        chunks.append(chunk)
    return chunks

In [64]:
from tqdm import tqdm

augmented_rows = []
for _, row in tqdm(dataset.iterrows(), total=len(dataset)):
    chunks = split_text_into_chunks(row.text, sentences_per_chunk=3)
    for chunk in chunks:
        augmented_rows.append({"text": chunk, "title": row.title})

augmented_dataset = pd.DataFrame(augmented_rows)

100%|██████████| 30623/30623 [00:29<00:00, 1036.64it/s]


In [65]:
augmented_dataset.describe()

,text,title
count,1682857,1682857
unique,1649981,28763
top,. . .,чистый хозяин собственного мира
freq,306,13958


In [67]:
augmented_dataset = augmented_dataset[augmented_dataset.text.str.strip().str.len() > 10]
augmented_dataset = augmented_dataset[augmented_dataset.text != ". . ."]

max_per_title = 1000
augmented_dataset = augmented_dataset.groupby("title").head(max_per_title).reset_index(drop=True)

print(augmented_dataset.title.value_counts().head(10))

title
тайна пяти печатей.                               1000
лекарство от старости                             1000
грибник                                           1000
небесный град (историко-фантастический роман)     1000
саркома                                           1000
подлунное  княжество                              1000
не могу молчать! вспоминая и переосмысливая...    1000
откровения каскадёра                              1000
завещание или почти бескорыстные мистификаторы    1000
пособие для начинающей проститутки                1000
Name: count, dtype: int64


In [68]:
augmented_dataset.describe()

,text,title
count,1313423,1313423
unique,1282668,28755
top,если же кого-то заинтересует более подробно то...,тайна пяти печатей.
freq,63,1000


In [70]:
augmented_dataset = augmented_dataset.drop_duplicates().reset_index(drop=True)

In [71]:
augmented_dataset.describe()

,text,title
count,1309490,1309490
unique,1282668,28755
top,"это не значит, что остальным жителям планеты н...",экологическая тропа
freq,63,1000


In [72]:
dup_titles = (
    augmented_dataset.groupby("text")["title"]
    .nunique()
    .reset_index()
    .query("title > 1")
)

print(f"Текстов с одинаковыми содержаниями, но разными названиями: {len(dup_titles)}")

Таких текстов: 5799


In [73]:
augmented_dataset = augmented_dataset.groupby("text").agg(lambda x: x.value_counts().index[0]).reset_index()

In [74]:
augmented_dataset.describe()

,text,title
count,1282668,1282668
unique,1282668,28735
top,"как любому живому организму, организации нуж...",сто блюд из кошки
freq,1,1000


In [77]:
def is_meaningful(text):
    return bool(re.search(r"[А-Яа-яA-Za-z0-9]", text))

In [78]:
augmented_dataset = augmented_dataset[augmented_dataset.text.apply(is_meaningful)].reset_index(drop=True)

In [79]:
augmented_dataset.describe()

,text,title
count,1281911,1281911
unique,1281911,28733
top,"как любому живому организму, организации нуж...",пандемия
freq,1,1000


In [80]:
augmented_dataset = augmented_dataset[augmented_dataset.text.str.len() > 10]
augmented_dataset = augmented_dataset[augmented_dataset.text.str.split().str.len() > 5]

In [82]:
augmented_dataset.reset_index(inplace=True, drop=True)

In [83]:
augmented_dataset.describe()

,text,title
count,1274717,1274717
unique,1274717,28650
top,"как любому живому организму, организации нуж...",откровения каскадёра
freq,1,1000


In [84]:
augmented_dataset.text = augmented_dataset.text.str.replace(r"[^\w\s,.!?-]", " ", regex=True)
augmented_dataset.text = augmented_dataset.text.str.replace(r"\s+", " ", regex=True).str.strip()

In [85]:
augmented_dataset.describe()

,text,title
count,1274717,1274717
unique,1274405,28650
top,привыкли добрыми. происшествие в моей нелегкой...,откровения каскадёра
freq,2,1000


In [92]:
def clean_leading_punct(text):
    return re.sub(r"^[^\wА-Яа-я0-9]+", "", text).strip()

In [93]:
augmented_dataset.text = augmented_dataset.text.apply(clean_leading_punct)

In [94]:
augmented_dataset.describe()

,text,title
count,1274093,1274093
unique,1274080,28600
top,"и пони, и дерьмо, и даже сигареты с шишкой на ...",пифагореец
freq,2,1000


In [95]:
augmented_dataset = augmented_dataset[~augmented_dataset.duplicated(subset=["text"], keep=False)]

In [96]:
augmented_dataset.describe()

,text,title
count,1274067,1274067
unique,1274067,28600
top,"как любому живому организму, организации нужна...",дорога цвета собаки
freq,1,1000


In [97]:
augmented_dataset.to_csv("../data/augmented_dataset.csv", index=False)